### Date Reformatting

In [1]:
import pickle
import pandas as pd

In [2]:
data_path = "./m_a_tkg/update_task/"

In [5]:
for f in ['all_tkg_snapshots_1010.pickle']:
    in_file = open(str(data_path + f), 'rb')
    all_tkg_snapshots = pickle.load(in_file)

In [6]:
a = all_tkg_snapshots[0]; a

[('apple_inc.',
  'owner of (P1830)',
  'Imagination Technologies (Q1119662)',
  '2018-01-01T00:00:00Z'),
 ('apple_inc.',
  'owner of (P1830)',
  'Apple corporate shuttle (Q110553619)',
  '2018-01-01T00:00:00Z'),
 ('apple_inc.',
  'subsidiary (P355)',
  'FileMaker, Inc. (Q1982831)',
  '2018-01-01T00:00:00Z'),
 ('apple_inc.',
  'subsidiary (P355)',
  'Anobit (Q2893391)',
  '2018-01-01T00:00:00Z'),
 ('apple_inc.',
  'subsidiary (P355)',
  'Beats Electronics (Q1961036)',
  '2018-01-01T00:00:00Z'),
 ('apple_inc.',
  'subsidiary (P355)',
  'Braeburn Capital (Q8084438)',
  '2018-01-01T00:00:00Z'),
 ('apple_inc.',
  'subsidiary (P355)',
  'AuthenTec (Q16835875)',
  '2018-01-01T00:00:00Z'),
 ('apple_inc.',
  'subsidiary (P355)',
  'FingerWorks (Q4038751)',
  '2018-01-01T00:00:00Z'),
 ('apple_inc.',
  'subsidiary (P355)',
  'Prismo Graphics (Q7245704)',
  '2018-01-01T00:00:00Z'),
 ('apple_inc.',
  'subsidiary (P355)',
  'Raycer (Q7298416)',
  '2018-01-01T00:00:00Z'),
 ('apple_inc.',
  'subsidia

In [8]:
last_state_tkg = pd.DataFrame(a, columns=['head_entity', 'relation', 'tail_entity', 'start_date'])

In [9]:
last_state_tkg

,head_entity,relation,tail_entity,start_date
0,apple_inc.,owner of (P1830),Imagination Technologies (Q1119662),2018-01-01T00:00:00Z
1,apple_inc.,owner of (P1830),Apple corporate shuttle (Q110553619),2018-01-01T00:00:00Z
2,apple_inc.,subsidiary (P355),"FileMaker, Inc. (Q1982831)",2018-01-01T00:00:00Z
3,apple_inc.,subsidiary (P355),Anobit (Q2893391),2018-01-01T00:00:00Z
4,apple_inc.,subsidiary (P355),Beats Electronics (Q1961036),2018-01-01T00:00:00Z
...,...,...,...,...
8314,sm_entertainment,subsidiary (P355),SM Classics (Q105753127),2018-01-01T00:00:00Z
8315,sm_entertainment,subsidiary (P355),Label V (Q107463954),2018-01-01T00:00:00Z
8316,sm_entertainment,subsidiary (P355),Baljunso (Q18159635),2018-01-01T00:00:00Z
8317,sm_entertainment,subsidiary (P355),SM Life Design Group (Q107473707),2018-01-01T00:00:00Z


In [10]:
all_entities = pd.Series(list(last_state_tkg['head_entity']) + list(last_state_tkg['tail_entity']))
v = all_entities.value_counts()

In [12]:
# split last_state_tkg into two tkgs
tkg_df_one = last_state_tkg[last_state_tkg['head_entity'].isin(v.index[v.eq(1)]) | last_state_tkg['tail_entity'].isin(v.index[v.eq(1)])]
tkg_df_gt_one = last_state_tkg.drop(tkg_df_one.index)

In [13]:
c = tkg_df_gt_one.groupby('head_entity', group_keys=False).apply(lambda df: df.sample(1))
d = tkg_df_gt_one.groupby('tail_entity', group_keys=False).apply(lambda df: df.sample(1))
e = sorted(set(list(c.index) + list(d.index)))
f = tkg_df_gt_one[tkg_df_gt_one.index.isin(e)]

In [14]:
# train-val-test split
train_valid_set = pd.concat([tkg_df_one, f])
test_set = tkg_df_gt_one[~tkg_df_gt_one.index.isin(e)]

In [16]:
len(train_valid_set)

7595

In [17]:
len(test_set)

724

In [18]:
train_valid_ents = pd.Series(list(train_valid_set['head_entity']) + list(train_valid_set['tail_entity']))

In [1]:
v = train_valid_ents.value_counts()

NameError: name 'train_valid_ents' is not defined

In [20]:
c = train_valid_set.groupby('head_entity', group_keys=False).apply(lambda df: df.sample(1))
d = train_valid_set.groupby('tail_entity', group_keys=False).apply(lambda df: df.sample(1))
e = sorted(set(list(c.index) + list(d.index)))

In [21]:
train_set = train_valid_set[train_valid_set.index.isin(e)]
valid_set = train_valid_set[~train_valid_set.index.isin(e)]

In [22]:
len(train_set)

7406

In [23]:
len(valid_set)

189

In [24]:
valid_test_set = pd.concat([valid_set, test_set])

In [26]:
from sklearn.model_selection import train_test_splits
new_valid_set, new_test_set = train_test_split(valid_test_set, train_size=0.5)

In [4]:
last_state_tkg = pd.DataFrame(all_tkg_snapshots[-1], columns=['head_entity', 'relation', 'tail_entity', 'start_date', 'end_date'])

In [5]:
last_state_tkg['new_start_date'] = last_state_tkg['start_date'].apply(lambda x: x.split(' ')[0])
last_state_tkg['new_end_date'] = last_state_tkg['end_date'].apply(lambda x: x.split(' ')[0] if type(x) == type("string") else x)

In [6]:
last_state_tkg['new_end_date'].isna().value_counts()

True     20120
False     1605
Name: new_end_date, dtype: int64

In [7]:
def get_be(begin, end):
    if type(begin) is type(None): # nan
#         begin = (-math.inf, 0, 0)
        begin = (2018, 1, 1)
    else:
        date_begin = begin.split('T')[0].split('-')
        begin = (int(date_begin[0]), int(date_begin[1]), int(date_begin[2]))

    if type(end) is type(None): # nan
#         end = (math.inf, 0, 0)
        end = (2021, 7, 1)
    else:
        date_end = end.split('T')[0].split('-')
        end = (int(date_end[0]), int(date_end[1]), int(date_end[2]))

    return begin, end

In [8]:
last_state_tkg['tkg_dates'] = last_state_tkg.apply(lambda x: get_be(x['new_start_date'], x['new_end_date']), axis = 1)

In [9]:
last_state_tkg['tkg_start_date'] = [x[0] for x in list(last_state_tkg['tkg_dates'])]
last_state_tkg['tkg_end_date'] = [x[1] for x in list(last_state_tkg['tkg_dates'])]

In [10]:
last_state_tkg.drop(['tkg_dates', 'new_start_date', 'new_end_date'], axis = 1, inplace = True)

In [11]:
last_state_tkg['end_date?'] = last_state_tkg['end_date'].apply(lambda x: False if type(x) == type(None) else True)

In [12]:
last_state_tkg

,head_entity,relation,tail_entity,start_date,end_date,tkg_start_date,tkg_end_date,end_date?
0,apple_inc.,owner of (P1830),Imagination Technologies (Q1119662),2018-01-01T00:00:00Z,None,"(2018, 1, 1)","(2021, 7, 1)",False
1,apple_inc.,owner of (P1830),Apple corporate shuttle (Q110553619),2018-01-01T00:00:00Z,None,"(2018, 1, 1)","(2021, 7, 1)",False
2,apple_inc.,subsidiary (P355),"FileMaker, Inc. (Q1982831)",2018-01-01T00:00:00Z,None,"(2018, 1, 1)","(2021, 7, 1)",False
3,apple_inc.,subsidiary (P355),Anobit (Q2893391),2018-01-01T00:00:00Z,None,"(2018, 1, 1)","(2021, 7, 1)",False
4,apple_inc.,subsidiary (P355),Beats Electronics (Q1961036),2018-01-01T00:00:00Z,None,"(2018, 1, 1)","(2021, 7, 1)",False
...,...,...,...,...,...,...,...,...
21720,tdg gold corp,expecting_acq,kingsgate_consolidated_limited,2021-06-29T21:30:00Z,None,"(2021, 6, 29)","(2021, 7, 1)",False
21721,tervita corporation,expecting_merger,tervita,2021-06-30T06:17:00Z,None,"(2021, 6, 30)","(2021, 7, 1)",False
21722,american_lithium_energy_(united_states),success_merger,plateau energy metals,2021-06-30T11:00:00Z,None,"(2021, 6, 30)","(2021, 7, 1)",False
21723,nxlv,success_acq,nexliving,2021-06-30T14:00:00Z,None,"(2021, 6, 30)","(2021, 7, 1)",False


### Train-Valid and Test set

In [13]:
all_entities = pd.Series(list(last_state_tkg['head_entity']) + list(last_state_tkg['tail_entity']))
v = all_entities.value_counts()

In [14]:
# split last_state_tkg into two tkgs
tkg_df_one = last_state_tkg[last_state_tkg['head_entity'].isin(v.index[v.eq(1)]) | last_state_tkg['tail_entity'].isin(v.index[v.eq(1)])]
tkg_df_gt_one = last_state_tkg.drop(tkg_df_one.index)

In [15]:
c = tkg_df_gt_one.groupby('head_entity', group_keys=False).apply(lambda df: df.sample(1))
d = tkg_df_gt_one.groupby('tail_entity', group_keys=False).apply(lambda df: df.sample(1))
e = sorted(set(list(c.index) + list(d.index)))
f = tkg_df_gt_one[tkg_df_gt_one.index.isin(e)]

In [16]:
# train-val-test split
train_valid_set = pd.concat([tkg_df_one, f])
test_set = tkg_df_gt_one[~tkg_df_gt_one.index.isin(e)]

### Train and Valid set

In [17]:
train_valid_ents = pd.Series(list(train_valid_set['head_entity']) + list(train_valid_set['tail_entity']))

In [18]:
v = train_valid_ents.value_counts()

In [19]:
c = train_valid_set.groupby('head_entity', group_keys=False).apply(lambda df: df.sample(1))
d = train_valid_set.groupby('tail_entity', group_keys=False).apply(lambda df: df.sample(1))
e = sorted(set(list(c.index) + list(d.index)))

In [20]:
train_set = train_valid_set[train_valid_set.index.isin(e)]
valid_set = train_valid_set[~train_valid_set.index.isin(e)]

### Rebalance the Valid and Test set

In [21]:
valid_test_set = pd.concat([valid_set, test_set])

In [22]:
from sklearn.model_selection import train_test_split
new_valid_set, new_test_set = train_test_split(valid_test_set, train_size=0.5, stratify=valid_test_set['end_date?'])

### Final Train-Valid-Test split

In [23]:
train_set['end_date?'].value_counts()

False    12146
True       831
Name: end_date?, dtype: int64

In [24]:
new_valid_set['end_date?'].value_counts()

False    3987
True      387
Name: end_date?, dtype: int64

In [25]:
new_test_set['end_date?'].value_counts()

False    3987
True      387
Name: end_date?, dtype: int64

In [26]:
train_set

,head_entity,relation,tail_entity,start_date,end_date,tkg_start_date,tkg_end_date,end_date?
0,apple_inc.,owner of (P1830),Imagination Technologies (Q1119662),2018-01-01T00:00:00Z,None,"(2018, 1, 1)","(2021, 7, 1)",False
1,apple_inc.,owner of (P1830),Apple corporate shuttle (Q110553619),2018-01-01T00:00:00Z,None,"(2018, 1, 1)","(2021, 7, 1)",False
2,apple_inc.,subsidiary (P355),"FileMaker, Inc. (Q1982831)",2018-01-01T00:00:00Z,None,"(2018, 1, 1)","(2021, 7, 1)",False
5,apple_inc.,subsidiary (P355),Braeburn Capital (Q8084438),2018-01-01T00:00:00Z,None,"(2018, 1, 1)","(2021, 7, 1)",False
6,apple_inc.,subsidiary (P355),AuthenTec (Q16835875),2018-01-01T00:00:00Z,None,"(2018, 1, 1)","(2021, 7, 1)",False
...,...,...,...,...,...,...,...,...
21671,hollywood's mgm studios,owned by (P127),Jeff Bezos (Q312556),2021-06-22T19:27:00Z,None,"(2021, 6, 22)","(2021, 7, 1)",False
21692,alliance healthcare services,board member (P3320),Fritz Blüthgen (Q95274925),2021-06-25T11:22:28Z,None,"(2021, 6, 25)","(2021, 7, 1)",False
21705,casa del caffè vergnano,board member (P3320),Marc Bolland (Q3030584),2021-06-28T17:40:00Z,None,"(2021, 6, 28)","(2021, 7, 1)",False
21707,casa del caffè vergnano,board member (P3320),Barry Diller (Q587455),2021-06-28T17:40:00Z,None,"(2021, 6, 28)","(2021, 7, 1)",False


In [27]:
train_set.to_csv('tkge_train_2609.csv', index = False)
new_valid_set.to_csv('tkge_valid_2609.csv', index = False)
new_test_set.to_csv('tkge_test_2609.csv', index = False)